James Fisher
ANA680
8/25/2024


                                    Week 3, Assignment 5 (SageMaker section)


* Part 1 of this ANA680 Week 3 SageMaker assignment uses code to build an ML project without container technology.

* Part 2 is below Part 1 and builds the model using container technology.

First, our INITIAL SETUP:

In [1]:
#installs
%pip install ucimlrepo scikit-learn joblib gunicorn

Note: you may need to restart the kernel to use updated packages.


In [2]:
#import libraries
import numpy as np
import pandas as pd

import os
import io
import joblib

from ucimlrepo import fetch_ucirepo
import sagemaker
import boto3
import flask
import gunicorn

import sklearn
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline, make_pipeline

from sklearn.linear_model import LinearRegression
from sagemaker import get_execution_role, LinearLearner, Session
from sagemaker import LinearLearner
from sagemaker.inputs import TrainingInput
from sagemaker.serializers import CSVSerializer
from sagemaker.deserializers import JSONDeserializer
from sagemaker.image_uris import retrieve
from sagemaker.amazon.common import write_numpy_to_dense_tensor
from sagemaker.sklearn.estimator import SKLearn
from sagemaker.sklearn.model import SKLearnModel

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


In [3]:
#library versions for requirements.txt
print("Scikit-Learn:", sklearn.__version__)
print("NumPy:", np.__version__)
print("Pandas:", pd.__version__)
print("Joblib:", joblib.__version__)
print("flask:", flask.__version__)
print("gunicorn", gunicorn.__version__)
print("SageMaker SDK:", sagemaker.__version__)
print("Boto3:", boto3.__version__)

import platform
print("Python:", platform.python_version())

Scikit-Learn: 1.5.1
NumPy: 2.1.0
Pandas: 2.2.2
Joblib: 1.4.2
flask: 3.0.3
gunicorn 23.0.0
SageMaker SDK: 2.229.0
Boto3: 1.35.2
Python: 3.10.14


/tmp/ipykernel_5905/2423915654.py:7: DeprecationWarning: The '__version__' attribute is deprecated and will be removed in Flask 3.1. Use feature detection or 'importlib.metadata.version("flask")' instead.
  print("flask:", flask.__version__)


In [5]:
#load wine quality data from UCI
wine_df = fetch_ucirepo(id=186)

X = wine_df.data.features
y = wine_df.data.targets

In [7]:
#confirm load
X.head

<bound method NDFrame.head of       fixed_acidity  volatile_acidity  citric_acid  residual_sugar  chlorides  \
0               7.4              0.70         0.00             1.9      0.076   
1               7.8              0.88         0.00             2.6      0.098   
2               7.8              0.76         0.04             2.3      0.092   
3              11.2              0.28         0.56             1.9      0.075   
4               7.4              0.70         0.00             1.9      0.076   
...             ...               ...          ...             ...        ...   
6492            6.2              0.21         0.29             1.6      0.039   
6493            6.6              0.32         0.36             8.0      0.047   
6494            6.5              0.24         0.19             1.2      0.041   
6495            5.5              0.29         0.30             1.1      0.022   
6496            6.0              0.21         0.38             0.8      0.020  

Split data into training/testing segments and save as .csv files for upload into S3

In [8]:
#split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=66)

In [9]:
#print shapes of train and test segments
print(f"X_train: {X_train.shape}")
print(f"y_train: {y_train.shape}\n")

print(f"X_test: {X_test.shape}")
print(f"y_test: {y_test.shape}")

X_train: (4872, 11)
y_train: (4872, 1)

X_test: (1625, 11)
y_test: (1625, 1)


In [10]:
#save training and test dfs as .csv files
X_train.to_csv("wine_train_features.csv", header=False, index=False)
y_train.to_csv("wine_train_labels.csv", header=False, index=False)
X_test.to_csv("wine_test_features.csv", header=False, index=False)
y_test.to_csv("wine_test_labels.csv", header=False, index=False)

Upload data to S3

In [11]:
#initialize SageMaker session
boto_session = boto3.Session(region_name='us-east-1')
session = sagemaker.Session(boto_session=boto_session)
bucket = session.default_bucket()
prefix = 'sagemaker/wine-quality'

#upload training data to S3
training_input_features_path = session.upload_data("wine_train_features.csv", bucket=bucket, key_prefix=f"{prefix}/train")
training_input_labels_path = session.upload_data("wine_train_labels.csv", bucket=bucket, key_prefix=f"{prefix}/train")

#upload test data to S3
test_input_features_path = session.upload_data("wine_test_features.csv", bucket=bucket, key_prefix=f"{prefix}/test")
test_input_labels_path = session.upload_data("wine_test_labels.csv", bucket=bucket, key_prefix=f"{prefix}/test")

In [14]:
%%writefile script.py

import os
import argparse
import joblib
import numpy as np
import pandas as pd

from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline

#define model function
def model_fn(model_dir):
    clf = joblib.load(os.path.join(model_dir, 'model.joblib'))
    return clf

if __name__ == '__main__':
    parser = argparse.ArgumentParser()
    
    #set SageMaker parameters
    parser.add_argument('--output-data-dir', type=str, default=os.environ['SM_OUTPUT_DATA_DIR'])
    parser.add_argument('--model-dir', type=str, default=os.environ['SM_MODEL_DIR'])
    parser.add_argument('--train_features', type=str, default=os.environ['SM_CHANNEL_TRAIN_FEATURES'])
    parser.add_argument('--train_labels', type=str, default=os.environ['SM_CHANNEL_TRAIN_LABELS'])

    args = parser.parse_args()
    
    #load datasets
    X_train = pd.read_csv(os.path.join(args.train_features, 'wine_train_features.csv'), header=None)
    y_train = pd.read_csv(os.path.join(args.train_labels, 'wine_train_labels.csv'), header=None)

    #convert to NumPy arrays
    X_train = X_train.to_numpy()
    y_train = y_train.to_numpy()

    #create pipeline with initial scaler
    model = make_pipeline(
        StandardScaler(),
        LinearRegression()
    )
    
    #train model
    model.fit(X_train, y_train)
    
    #save model
    joblib.dump(model, os.path.join(args.model_dir, 'model.joblib'))

Writing script.py


In [16]:
#define SKLearn estimator
role = sagemaker.get_execution_role()
sklearn_estimator = SKLearn(entry_point='script.py',
                            role=role,
                            instance_type='ml.m4.xlarge',
                            framework_version='1.2-1',
                            py_version='py3',
                            script_mode=True,
                            sagemaker_session=session)

#fit model
sklearn_estimator.fit({'train_features': training_input_features_path, 'train_labels': training_input_labels_path})

INFO:sagemaker:Creating training-job with name: sagemaker-scikit-learn-2024-08-25-19-50-40-969


2024-08-25 19:50:42 Starting - Starting the training job...
2024-08-25 19:50:56 Starting - Preparing the instances for training...
2024-08-25 19:51:23 Downloading - Downloading input data...
2024-08-25 19:51:53 Downloading - Downloading the training image......
2024-08-25 19:53:14 Training - Training image download completed. Training in progress.
2024-08-25 19:53:14 Uploading - Uploading generated training model2024-08-25 19:53:05,250 sagemaker-containers INFO     Imported framework sagemaker_sklearn_container.training
2024-08-25 19:53:05,253 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2024-08-25 19:53:05,256 sagemaker-training-toolkit INFO     No Neurons detected (normal if no neurons installed)
2024-08-25 19:53:05,271 sagemaker_sklearn_container.training INFO     Invoking user training script.
2024-08-25 19:53:05,484 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2024-08-25 19:53:05,488 sagemaker-training-too

Model Deployment

In [17]:
#build model from artifacts
sklearn_estimator.latest_training_job.wait(logs='None')

artifact = boto3.client('sagemaker', region_name='us-east-1').describe_training_job(
    TrainingJobName=sklearn_estimator.latest_training_job.name
)['ModelArtifacts']['S3ModelArtifacts']

print(f"Model artifact persisted at {artifact}")


2024-08-25 19:53:23 Starting - Preparing the instances for training
2024-08-25 19:53:23 Downloading - Downloading the training image
2024-08-25 19:53:23 Training - Training image download completed. Training in progress.
2024-08-25 19:53:23 Uploading - Uploading generated training model
2024-08-25 19:53:23 Completed - Training job completed
Model artifact persisted at s3://sagemaker-us-east-1-495599760214/sagemaker-scikit-learn-2024-08-25-19-50-40-969/output/model.tar.gz


In [18]:
#initialize model
model = SKLearnModel(
    model_data=artifact,
    role=get_execution_role(),
    entry_point='script.py',
    framework_version='1.2-1',
    sagemaker_session=session
)

In [20]:
#deploy initalized model
predictor = model.deploy(initial_instance_count=1, instance_type='ml.m4.xlarge')

INFO:sagemaker:Creating model with name: sagemaker-scikit-learn-2024-08-25-20-33-29-967
INFO:sagemaker:Creating endpoint-config with name sagemaker-scikit-learn-2024-08-25-20-33-30-570
INFO:sagemaker:Creating endpoint with name sagemaker-scikit-learn-2024-08-25-20-33-30-570


-------!

In [21]:
#make predictions on test dataframe
predictions = predictor.predict(X_test.to_numpy())

#evaluate model performance
mse = mean_squared_error(y_test, predictions)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y_test, predictions)
r2 = r2_score(y_test, predictions)

metrics_df = pd.DataFrame({
    'Metric': ['MSE', 'RMSE', 'MAE', 'R2'],
    'Value': [mse, rmse, mae, r2]
})
metrics_df

,Metric,Value
0,MSE,0.502573
1,RMSE,0.708924
2,MAE,0.548648
3,R2,0.318938


In [22]:
# END OF PART 1. Cleaned up Endpoints prior to beginning Part 2 (model deployment using a container).
predictor.delete_endpoint()

INFO:sagemaker:Deleting endpoint configuration with name: sagemaker-scikit-learn-2024-08-25-20-33-30-570
INFO:sagemaker:Deleting endpoint with name: sagemaker-scikit-learn-2024-08-25-20-33-30-570


PART 2. Deploying with a Container.

In [26]:
#get local directory
import os
current_directory = os.getcwd()
print("Current working directory:", current_directory)

Current working directory: /home/ec2-user/SageMaker


In [28]:
#download trained model

import tarfile

#initialize a S3 client
s3_client = boto3.client('s3')

#set S3 path (from Training Jobs output details)
artifact_path = 's3://sagemaker-us-east-1-495599760214/sagemaker-scikit-learn-2024-08-25-19-50-40-969/output/model.tar.gz'

#parse the S3 path to get bucket and object key
bucket, key = artifact_path.replace("s3://", "").split("/", 1)

#define local path to download the model.tar.gz file
local_tar_path = '/home/ec2-user/SageMaker/model.tar.gz'

#download model artifact from S3 to local path
s3_client.download_file(bucket, key, local_tar_path)

#extract the tar.gz file
with tarfile.open(local_tar_path) as tar:
    tar.extractall(path='/home/ec2-user/SageMaker')

print("Model artifact downloaded and extracted.")

Model artifact downloaded and extracted.


In [31]:
import warnings
from sklearn.exceptions import InconsistentVersionWarning
warnings.filterwarnings("ignore", category=InconsistentVersionWarning)

#load model from .joblib file
model = joblib.load('/home/ec2-user/SageMaker/model.joblib')

#save model as a .pkl file
joblib.dump(model, '/home/ec2-user/SageMaker/model.pkl')

print("Model saved as .pkl file.")

Model saved as .pkl file.


In [33]:
#print model
print(model)

#print parameters
print(model.get_params())

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('linearregression', LinearRegression())])
{'memory': None, 'steps': [('standardscaler', StandardScaler()), ('linearregression', LinearRegression())], 'verbose': False, 'standardscaler': StandardScaler(), 'linearregression': LinearRegression(), 'standardscaler__copy': True, 'standardscaler__with_mean': True, 'standardscaler__with_std': True, 'linearregression__copy_X': True, 'linearregression__fit_intercept': True, 'linearregression__n_jobs': None, 'linearregression__positive': False}


Save docker image to ECL and Deploy

In [38]:
!docker build -t wine-quality-predictor .

[+] Building 0.0s (0/1)                                          docker:default
[+] Building 0.1s (3/10)                                         docker:default
 => [internal] load build definition from Dockerfile                       0.0s
 => => transferring dockerfile: 962B                                       0.0s
 => [internal] load metadata for docker.io/library/python:3.10-slim        0.1s
 => [internal] load .dockerignore                                          0.0s
 => => transferring context: 2B                                            0.0s
[+] Building 0.3s (7/10)                                         docker:default
 => [internal] load build definition from Dockerfile                       0.0s
 => => transferring dockerfile: 962B                                       0.0s
 => [internal] load metadata for docker.io/library/python:3.10-slim        0.1s
 => [internal] load .dockerignore                                          0.0s
 => => transferring context: 2B         

In [39]:
!aws ecr get-login-password --region us-east-1 | docker login --username AWS --password-stdin 495599760214.dkr.ecr.us-east-1.amazonaws.com

WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store

Login Succeeded


In [40]:
!docker tag wine-quality-predictor:latest 495599760214.dkr.ecr.us-east-1.amazonaws.com/ana680-wk3/wine-quality-predictor:latest

In [42]:
!docker push 495599760214.dkr.ecr.us-east-1.amazonaws.com/ana680-wk3/wine-quality-predictor:latest

The push refers to repository [495599760214.dkr.ecr.us-east-1.amazonaws.com/ana680-wk3/wine-quality-predictor]

e06d8ca6: Preparing 
885151fa: Preparing 
2f81d733: Preparing 
ea06793d: Preparing 
4d3eed49: Preparing 
99102447: Preparing 
8766068e: Preparing 
b2e74780: Preparing 
ca8b8119: Preparing 
2f81d733: Pushed   408.9MB/395.8MBPushing  216.5MB/395.8MBlatest: digest: sha256:2f0f943365f6303f75952713bb4823335bb2cb17834bb104f4ee33895001001e size: 2412


In [43]:
import sagemaker
from sagemaker.sklearn.model import SKLearnModel
from sagemaker import get_execution_role

#define ECR path to the Docker image
image_uri = '495599760214.dkr.ecr.us-east-1.amazonaws.com/ana680-wk3/wine-quality-predictor:latest'

#get the SageMaker execution role
role = get_execution_role()

#create SageMaker model
model = SKLearnModel(
    model_data=artifact_path,
    role=role,
    entry_point='serve.py',
    framework_version='1.2-1',
    image_uri=image_uri,
    sagemaker_session=session
)

#deploy model to endpoint
predictor = model.deploy(
    initial_instance_count=1,
    instance_type='ml.m4.xlarge',
    endpoint_name='wine-quality-predictor-endpoint-final'
)

print(f'Model deployed to endpoint: {predictor.endpoint_name}')

INFO:sagemaker:Creating model with name: wine-quality-predictor-2024-08-25-21-38-18-302
INFO:sagemaker:Creating endpoint-config with name wine-quality-predictor-endpoint-final
INFO:sagemaker:Creating endpoint with name wine-quality-predictor-endpoint-final


----!Model deployed to endpoint: wine-quality-predictor-endpoint-final


Testing Endpoint

In [44]:
import json

runtime = boto3.client('runtime.sagemaker', region_name='us-east-1')

#define mock data payload
payload = {
    "inputs": [[5.2, 0.77, 0.03, 2.1, 0.065, 22.0, 61.0, 0.9522, 3.7, 0.57, 8.9]]
}

#invoke endpoint
response = runtime.invoke_endpoint(
    EndpointName='wine-quality-predictor-endpoint-final',
    ContentType='application/json',
    Body=json.dumps(payload)
)

#parse and return response
response_body = json.loads(response['Body'].read().decode('utf-8'))
print(f"Predicted Wine Quality: {response_body[0][0]}")

Predicted Wine Quality: 7.204453635528657


BOOYAH!!!!!!!!   Very, very cool.